This is the secondary area of code development; this will provide more detail and explanations around specific objects and functions within the main code body

Goal of code:
To provide a GUI to support the creation of an independence risk RAG for people entering social care and those having thier care packages reviewed by entering a few key details.

This will support more proactive engagement to promote increased independence over a longer time period

Key Libraries: Thsi tool is dependent on the following modules

In [ ]:
#import key libraries
import pandas as pd
import tkinter as tk
import datetime as dt
import math
import numpy as np

Service information dataframe: This provides key information regarding services and impact on risk modifiers; created as a dataframe to facilitate easy refereing elsewhere in the code

In [ ]:
#create a dataframe of each service variety, intensity level and risk factor modifier
#create lists
ServType = ["Equipment","Day Support","Direct Payment","Homecare:Low","Homecare:Mid","Homecare:High"]
ServIntens = [1,2,3,4,5,6]
ServModify = [0.6,0.8,1,1.1,1.3,1.5]
Header = ["Service","Intensity","RiskModifier"]
#combine lists to array
ServInfo = np.array((ServType))
ServInfo=ServInfo.reshape(6,1)
ServInfo = np.hstack((ServInfo,np.array((ServIntens)).reshape(6,1),np.array((ServModify)).reshape(6,1)))
#convert array to dataframe
ServInfo = pd.DataFrame(ServInfo,columns = Header)


AssessRev Class Code
This is the class parameters for the AssessRev class, which is the primary object for this tool; each person has an instance within this class that stores thier current RAG status

In [ ]:
class AssessRev:
    """This contains details of individuals at the point of a review or assessment that underpin the creation of a non-independent care risk rating
    Methods:
        update_AgeFac()
        update_ServFac()
        update_ServChange()
        update_RAG()
    Attributes:
        PersonId - This is the unique identification number for the person recieving the assessment or review. Required to be an integer
        ContactDate - The date that the review or assessment was completed
        BirthDate - The person recieving the reviews date of birth
        CurrentServ - This is the service currently in place; defined list of options
        NewServ - This is the service that will be in place following the review; defined list, shared with CurrentServ
        BaseRisk - The starting risk score for calculating RAG. Default = 1
        AgeFac - The risk modification factor associated with the persons age band.  Default = 1
        ServFac - The risk modification factor associated with the service type in NewServ. Default = 1
        ServChange - The risk modification factor associated with the direction of chnage between CurrentServ and NewServ.  Default = 1
        RAG - The combined risk score of the person entering non-independent care within the next 12 months, Default = 1
    """
    #create initialisation method to include key details at initialisation
    def __init__(self,PersonId,ContactDate,BirthDate,CurrentServ,NewServ,BaseRisk=1,AgeFac=1,ServFac=1,ServChange=1,RAG=1):
        self.PersonId = PersonId
        self.ContactDate = pd.to_datetime(ContactDate,dayfirst=True)
        self.BirthDate = pd.to_datetime(BirthDate,dayfirst=True)
        self.CurrentServ = CurrentServ
        self.NewServ = NewServ
        self.BaseRisk = BaseRisk
        self.AgeFac = AgeFac
        self.ServFac = ServFac
        self.ServChange=ServChange
        self.RAG = RAG

    #create method to update AgeFac
    def update_AgeFac(self,AgeFacNew):
        """This method allows the replacement of the existing AgeFac of an AssessRev instance with a new value"""
        self.AgeFac = AgeFacNew
    
    #create method to update ServFac
    def update_ServFac(self,ServFacNew):
        """This method allows the replacement of the existing ServFac of an AssessRev instance with a new value"""
        self.ServFac = ServFacNew

    #create method to update ServChange
    def update_ServChange(self,ServChangeNew):
        """This method allows the replacement of the existing ServChange of an AssessRev instance with a new value"""
        self.ServChange = ServChangeNew

    #create method to update RAG
    def update_RAG(self,RagNew):
        """This method allows the replacement of the existing RAG of an AssessRev instance with a new value"""
        self.RAG=RagNew
      
    pass